In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('data/datasets/ais_train.csv', sep='|')
train['time'] = pd.to_datetime(train['time'])
train.info()
missing_values = train.isnull().sum()
print("Number of missing values in each column:\n", missing_values)

train.head()

test = pd.read_csv('data/datasets/ais_test.csv', sep=',')
test['time'] = pd.to_datetime(test['time'])
test.head()

vessels = pd.read_csv('data/datasets/vessels.csv', sep='|')
vessels.head()

ports = pd.read_csv('data/datasets/ports.csv', sep='|')
ports.head()

In [ ]:
train = train.dropna(subset=['portId'])
print(train.shape)
# Verify the result by checking the count of missing values
print("Number of missing values in each column after dropping rows with NaN in 'portId':")
print(train.isna().sum())

In [ ]:
schedules = pd.read_csv('data/datasets/schedules_to_may_2024.csv', sep='|')

In [ ]:
port_id_mapping = {port_id: idx for idx, port_id in enumerate(train['portId'].unique())}
train['portId'] = train['portId'].map(port_id_mapping)
ports['portId'] = ports['portId'].map(port_id_mapping)
schedules['portId'] = schedules['portId'].map(port_id_mapping)

vessel_id_mapping = {vessel_id: idx for idx, vessel_id in enumerate(train['vesselId'].unique())}
train['vesselId'] = train['vesselId'].map(vessel_id_mapping)
vessels['vesselId'] = vessels['vesselId'].map(vessel_id_mapping)
test['vesselId'] = test['vesselId'].map(vessel_id_mapping)
schedules['vesselId'] = schedules['vesselId'].map(vessel_id_mapping)


shipping_line_id_mapping = {shipping_line_id: idx for idx, shipping_line_id in enumerate(vessels['shippingLineId'].unique())}
vessels['shippingLineId'] = vessels['shippingLineId'].map(shipping_line_id_mapping)
schedules['shippingLineId'] = schedules['shippingLineId'].map(shipping_line_id_mapping)

In [1]:
import pytz
schedules['sailingDate'] = pd.to_datetime(schedules['sailingDate'], errors='coerce')
schedules['arrivalDate'] = pd.to_datetime(schedules['arrivalDate'], errors='coerce')

# Convert sailingDate and arrivalDate to a different timezone, such as CET (Central European Time)
# Convert datetime columns to timezone-naive format by removing timezone info
schedules['sailingDate'] = schedules['sailingDate'].dt.tz_convert(None)
schedules['arrivalDate'] = schedules['arrivalDate'].dt.tz_convert(None)


schedules.head()


NameError: name 'pd' is not defined

In [ ]:
vessels.head()

In [ ]:


train = train.merge(ports[['portId', 'latitude', 'longitude']], how='left', left_on='portId', right_on='portId', suffixes=('', '_port'))
train.head()

In [ ]:
train = train.merge(vessels[['vesselId', 'length', 'shippingLineId', 'breadth', 'GT']], on='vesselId', how='left')
train['vessel_deep_sea'] = np.where(train['length'] > 200, 1, 0)

num_maxSpeed_nan = train['breadth'].isna().sum()
print(f"Number of NaN values in maxSpeed: {num_maxSpeed_nan}")
train.head()

In [ ]:
# Calculate the mean breadth for shipping line 5
mean_breadth_shipping_line_5 = train[train['shippingLineId'] == 5]['breadth'].mean()

# Impute missing breadth with the calculated mean
train.loc[train['shippingLineId'] == 5, 'breadth'] = train.loc[train['shippingLineId'] == 5, 'breadth'].fillna(mean_breadth_shipping_line_5)

# Verify if the missing values were filled
missing_count_after_imputation = train['breadth'].isnull().sum()
print(f"Missing Breadth values after imputation: {missing_count_after_imputation}")


In [ ]:
train.info()
missing_values = train.isnull().sum()
print("Number of missing values in each column:\n", missing_values)

In [ ]:
df_days_before_prediction = train.copy()
# Ensure 'time' column is in datetime format
df_days_before_prediction['time'] = pd.to_datetime(df_days_before_prediction['time'])


# Specify the year to append
current_year = 2024  # Change to the desired year

# Modify etaRaw to include the year
df_days_before_prediction['etaRaw'] = df_days_before_prediction['etaRaw'].apply(lambda x: f"{current_year}-{x}")


# Convert 'etaRaw' column to datetime
df_days_before_prediction['etaRaw'] = pd.to_datetime(df_days_before_prediction['etaRaw'], format='%Y-%m-%d %H:%M', errors='coerce')

# Define the start of the prediction period and calculate minimum required date
prediction_start_date = df_days_before_prediction['time'].max()
days_before_start = 0  # Set to the number of days you want before the start date
min_required_date = prediction_start_date 

# Get the last recorded date for each vessel along with `etaRaw`
vessel_last_dates = df_days_before_prediction.groupby('vesselId').agg({'time': 'max', 'etaRaw': 'last'}).reset_index()
vessel_last_dates.rename(columns={'time': 'last_train_time'}, inplace=True)

# Check the minimum required date
print("\nMinimum required date:")
print(min_required_date)

# Add a new column to indicate if etaRaw is before the minimum required date
vessel_last_dates['etaRaw_before_required'] = vessel_last_dates['etaRaw'] < min_required_date

# Calculate days missing from the required date
vessel_last_dates['days_from_required'] = (min_required_date - vessel_last_dates['last_train_time']).dt.days

# Add a new column to indicate if the data is before the required date
vessel_last_dates['before_required_date'] = vessel_last_dates['days_from_required'] > 1

# Separate vessels with incomplete data
vessels_with_incomplete_data = vessel_last_dates[vessel_last_dates['before_required_date']]

# Plot vessels with incomplete data
plt.figure(figsize=(10, 6))
plt.barh(vessels_with_incomplete_data['vesselId'], vessels_with_incomplete_data['days_from_required'], color='orange')
plt.xlabel('Days from Required Date')
plt.ylabel('Vessel ID')
plt.title('Days Missing from Required Prediction Period by Vessel')
plt.gca().invert_yaxis()  # Invert y-axis to have the vessel with the most missing days on top
plt.grid(axis='x')  # Add gridlines for better readability
plt.show()



In [ ]:
# Define the minimum required date for predictions in the test set
min_required_date = test['time'].min() - pd.Timedelta(days=5)  # 5 days before the earliest test date
print(f"Minimum required date for predictions: {min_required_date}")

# Filter the schedules DataFrame based on the required date
schedules_filtered = schedules[~((schedules['sailingDate'] < min_required_date) & 
                                  (schedules['arrivalDate'] < min_required_date))]

# Get the last recorded date for each vessel in the training set
last_train_dates = train.groupby('vesselId')['time'].max().reset_index()
last_train_dates.rename(columns={'time': 'last_train_time'}, inplace=True)

# Print the filtered schedules for verification
# print("Filtered schedules:")
# print(schedules_filtered.head())

# Filter vessels that are in the test set
vessels_in_test = test['vesselId'].unique()
missing_days_df = last_train_dates[last_train_dates['vesselId'].isin(vessels_in_test)].copy()

# Calculate the days missing for each vessel from the required date
missing_days_df['days_missing'] = (min_required_date - missing_days_df['last_train_time']).dt.days

# Filter vessels missing a significant number of days (e.g., more than 2 days)
significant_gap_vessels = missing_days_df[missing_days_df['days_missing'] > 1]

# Print the vessels with significant gaps
print("Vessels in the test set missing significant data before prediction date:")
print(significant_gap_vessels[['vesselId', 'last_train_time', 'days_missing']])
print(f"Number of vessels with significant missing days: {len(significant_gap_vessels)}")

# Optional: If you want to check which vessels are in filtered schedules
vessels_in_filtered_schedules = schedules_filtered['vesselId'].unique()
print(f"Number of unique vessels in filtered schedules: {len(vessels_in_filtered_schedules)}")

# Step to find which significant gap vessels are in filtered schedules
vessels_in_gap_and_schedule = set(significant_gap_vessels['vesselId']).intersection(vessels_in_filtered_schedules)

# Create a DataFrame for common vessels with significant gaps in schedules
vessels_in_gap_and_schedule_df = significant_gap_vessels[significant_gap_vessels['vesselId'].isin(vessels_in_gap_and_schedule)]


# Print results
print("Vessels with significant gaps that are also in the filtered schedules:")
print(vessels_in_gap_and_schedule_df[['vesselId', 'last_train_time', 'days_missing']])
print(f"Number of vessels with significant missing days that are in schedules: {len(vessels_in_gap_and_schedule_df)}")

In [ ]:

# Step to find which significant gap vessels are NOT in filtered schedules
vessels_in_gap_not_in_schedule = set(significant_gap_vessels['vesselId']).difference(vessels_in_filtered_schedules)

# Create a DataFrame for vessels with significant gaps that are not in the schedules
vessels_in_gap_not_in_schedule_df = significant_gap_vessels[significant_gap_vessels['vesselId'].isin(vessels_in_gap_not_in_schedule)]

# Print results
print("Vessels with significant gaps that are NOT in the filtered schedules:")
print(vessels_in_gap_not_in_schedule_df[['vesselId', 'last_train_time', 'days_missing']])
print(f"Number of vessels with significant missing days that are NOT in schedules: {len(vessels_in_gap_not_in_schedule_df)}")

In [ ]:
schedules_filtered.head()

In [ ]:

def create_five_day_windows(df):
    # Ensure that 'time' column is in datetime format
    df['time'] = pd.to_datetime(df['time'])
    
    # Sort data to ensure time sequence within each vessel
    df = df.sort_values(by=['vesselId', 'time']).reset_index(drop=True)
    
    window_size_days = 5
    windows = []

    for vessel_id, group in df.groupby('vesselId'):
        group = group.sort_values(by='time')
        unique_dates = group['time'].dt.date.unique()

        for start_idx in range(len(unique_dates) - window_size_days + 1):
            start_date = unique_dates[start_idx]
            end_date = unique_dates[start_idx + window_size_days - 1]
            window = group[(group['time'].dt.date >= start_date) & (group['time'].dt.date <= end_date)]

            # Skip empty windows
            if len(window) == 0:
                continue

            # Reference row: the first row in the window
            reference_row = window.iloc[0]

            # Append each row in this window with reference features from the first row
            for _, row in window.iterrows():
                windows.append({
                    'vesselId': row['vesselId'],
                    'time': row['time'],
                    'latitude': row['latitude'],
                    'longitude': row['longitude'],
                    'cog_sin': row['cog_sin'],
                    'cog_cos': row['cog_cos'],
                    'cog' : row['cog'],
                    'sog': row['sog'],
                    'rot': row['rot'],
                    'under_way': row['under_way'],
                    'length' : row['length'],
                    'breadth': row['breadth'],
                    # 'DWT': row['DWT'],
                    'GT': row['GT'],
                    # 'vessel_deep_sea': row['vessel_deep_sea'],
                    # Features based on the first row in the window
                    'latitude_first': reference_row['latitude'],
                    'longitude_first': reference_row['longitude'],
                    'cog_sin_first': reference_row['cog_sin'],
                    'cog_cos_first': reference_row['cog_cos'],
                    'sog_first': reference_row['sog'],
                    'rot_first': reference_row['rot'],
                    'under_way_first': reference_row['under_way'],
                    'time_since_start': (row['time'] - reference_row['time']).total_seconds()
                })

    return pd.DataFrame(windows)

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)) 
    r = 6371  # Radius of Earth in kilometers
    return r * c  # Distance in kilometers

In [ ]:
from shapely.geometry import MultiPoint

def calculate_convex_hull_area(lats, lons):
    """Calculate the area of the convex hull formed by the given latitudes and longitudes."""
    # Create a list of (longitude, latitude) points
    points = [(lon, lat) for lon, lat in zip(lons, lats)]
    
    # If there are fewer than 3 unique points, the area is zero
    if len(points) < 3 or len(set(points)) < 3:
        return 0.0

    # Use MultiPoint to calculate the convex hull area
    multipoint = MultiPoint(points)
    hull = multipoint.convex_hull
    
    # Return the area of the convex hull
    return hull.area

In [ ]:
def future_position(lat, lon, cog, sog, time_diff):
    """
    Calculate the future position given the current latitude, longitude, COG, SOG, and time difference in seconds.
    """
    # Convert COG from degrees to radians
    cog_rad = np.radians(cog)

    # Calculate distance traveled in meters
    distance = sog * time_diff  # distance = speed * time

    # Calculate future latitude and longitude
    future_lat = lat + (distance * np.cos(cog_rad)) / 111320  # 111320 meters per degree latitude
    future_lon = lon + (distance * np.sin(cog_rad)) / (111320 * np.cos(np.radians(lat)))  # adjust for longitude

    return future_lat, future_lon

In [ ]:
def feature_engineering(train):
    train = train.copy()
    train = train.sort_values(by=['vesselId', 'time']).reset_index(drop=True)

    # # Handle NaNs
    # train[['latitude', 'longitude', 'cog', 'time']] = (
    #     train.groupby('vesselId')[['latitude', 'longitude', 'cog', 'time']].apply(lambda x: x.ffill().bfill())
    # ).reset_index(drop=True)

    features = pd.DataFrame()
    features['vesselId'] = train['vesselId']
    features['time'] = train['time'] 
    features['latitude'] = train['latitude']
    features['longitude'] = train['longitude']
    features['cog'] = train['cog']
    features['sog'] = train['sog']
    features['rot'] = train['rot']
    features['under_way'] = train['navstat'].isin([0, 8]).astype(int)
    features['cog_rad'] = np.radians(features['cog'])
    features['cog_sin'] = np.sin(features['cog_rad'])
    features['cog_cos'] = np.cos(features['cog_rad'])
    features['length'] = train['length']
    features['breadth'] = train['breadth']
    features['GT'] = train['GT']
    
    features = create_five_day_windows(features)

    features['latitude_port'] = train['latitude_port']
    features['longitude_port'] = train['longitude_port']

    features['distance_to_port'] = haversine(features['latitude'], features['longitude'], 
                                              features['latitude_port'], features['longitude_port'])
    
#     features['time_diff'] = features.groupby('vesselId')['time'].diff().dt.total_seconds().fillna(0)

#     # # Calculate future positions
#     # future_positions = features.apply(lambda row: future_position(
#     #     row['latitude'], row['longitude'], row['cog'], row['sog'], row['time_diff']), axis=1)

#     # features['future_latitude'], features['future_longitude'] = zip(*future_positions)
    
#     features['area_covered'] = features.apply(lambda row: calculate_convex_hull_area(
#         features[(features['vesselId'] == row['vesselId']) & (features['time'] <= row['time'])]['latitude'].tolist(),
#         features[(features['vesselId'] == row['vesselId']) & (features['time'] <= row['time'])]['longitude'].tolist()), axis=1)

#     # Calculate cumulative area covered for each vessel
#     features['cumulative_area_covered'] = features.groupby('vesselId')['area_covered'].cumsum()
 
    # Additional time-based features
    features['month'] = features['time'].dt.month
    features['day'] = features['time'].dt.day
    features['hour'] = features['time'].dt.hour
    features['minute'] = features['time'].dt.minute
    # features['day_of_week'] = features['time'].dt.dayofweek

    return features
train_features = feature_engineering(train)

In [ ]:
nan_values = train_features.isnull().sum()
print(nan_values)


# Drop the columns 'time' and 'rot'
train_features = train_features.drop(columns=['time', 'rot_first'])

In [ ]:
train_features = train_features.dropna()

train_features.tail()

In [ ]:
def find_last_features(features):
    # Assuming that the last row for every vesselId is the most recent
    last_features = features.groupby('vesselId').last().reset_index()
    return last_features

last_features = find_last_features(train_features)
# last_features.drop(columns=['DWT']) 
last_features.head(50)

In [ ]:
# # Define features and target
y = train_features[['latitude', 'longitude']]  
X = train_features.drop(columns=['latitude', 'longitude']) 
X.head()

In [ ]:
# best_model = RandomForestRegressor(max_depth=50, random_state=42)

# # Define the parameter grid for Random Forest
# param_grid = {
#     'n_estimators': [100, 300],            # Number of trees in the forest
#     'max_depth': [5, 10, 25, None],             # Maximum depth of the tree
#     'min_samples_split': [2, 5],           # Minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2],            # Minimum number of samples required to be at a leaf node
#     'max_features': ['auto', 'sqrt'],      # The number of features to consider when looking for the best split
#     'bootstrap': [True, False]             # Whether bootstrap samples are used when building trees
# }

# # Perform Randomized Search
# random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
#                                    n_iter=20, scoring='neg_mean_squared_error', cv=5, verbose=1, random_state=42)
# random_search.fit(X, y)

# best_model = random_search.best_estimator_

# # Output the best parameters and score
# print(f"Best parameters: {random_search.best_params_}")
# print(f"Best score (negative mean squared error): {-random_search.best_score_}")

best_model = xgb.XGBRegressor()

# Fit the model
best_model.fit(X, y)


In [ ]:

# Step 1: Get feature importances
feature_importances = best_model.feature_importances_

# Create a DataFrame to display feature importances
features_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort by importance
features_df = features_df.sort_values(by='Importance', ascending=False)

# Step 2: Print feature importances
print("Feature Importances:")
print(features_df)

In [ ]:

# Step 1: Get feature importances
feature_importances = best_model.feature_importances_

# Create a DataFrame to display feature importances
features_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort by importance
features_df = features_df.sort_values(by='Importance', ascending=False)

# Step 2: Print feature importances
print("Feature Importances:")
print(features_df)

In [ ]:
# Confirm unique vesselIds in vessels_in_gap_in_schedule
print("Unique vesselIds in vessels_in_gap_in_schedule:")
print(vessels_in_gap_and_schedule_df['vesselId'].unique())
print(f"Count of unique vesselIds: {len(vessels_in_gap_and_schedule_df['vesselId'].unique())}")




In [ ]:
matching_vessels = set(last_features['vesselId']).intersection(set(vessels_in_gap_and_schedule_df['vesselId']))
print(f"Number of matching vesselIds: {len(matching_vessels)}")


In [ ]:
last_features_gap = last_features[last_features['vesselId'].isin(vessels_in_gap_not_in_schedule_df['vesselId'])].copy()
last_features_gap.head()

last_features_gap['under_way'] = 0  # Stationary mode
last_features_gap['sog'] = 0        # Speed over ground to 0
last_features_gap['rot'] = 0        # Rate of turn to 0
last_features_gap['cog'] = 0        # Optional: course over ground to 0
last_features_gap['cog_sin'] = 0    # Sine of course to 0
last_features_gap['cog_cos'] = 1    # Cosine of course to 1
last_features_gap['distance_to_port'] = 0
last_features_gap['time_since_start'] = 0


# Align with the maximum time and date fields in last_features
last_features_gap['day'] = 6 #sunday before 8th of May 
last_features_gap['month'] = last_features['month'].max()
# last_features_gap['minute'] = last_features['minute'].max()
last_features_gap = last_features_gap.drop(columns=['year'], errors='ignore')

# Explicitly cast each column in last_features_gap to match last_features data types
for column in last_features_gap.columns:
    if column in last_features.columns:
        last_features_gap[column] = last_features_gap[column].astype(last_features[column].dtype)


last_features_gap.columns
last_features_gap.head()

In [ ]:
for column in last_features.columns:
    if column in last_features_gap.columns:
        last_features_gap[column] = last_features_gap[column].astype(last_features[column].dtype)

# Set `vesselId` as the index for both DataFrames to ensure alignment on `vesselId`
last_features.set_index('vesselId', inplace=True)
last_features_gap.set_index('vesselId', inplace=True)

# Update existing rows in `last_features` with values from `last_features_gap`
last_features.update(last_features_gap)

# Use `combine_first` to add any missing `vesselId`s from `last_features_gap` to `last_features`
last_features = last_features.combine_first(last_features_gap)

# Reset the index to restore `vesselId` as a column
last_features.reset_index(inplace=True)
last_features_gap.reset_index(inplace=True)

# Check a sample to verify the updates
print(last_features[last_features['vesselId'].isin(vessels_in_gap_not_in_schedule_df['vesselId'])].head())

In [ ]:
# Display rows with any NaN values in last_features
nan_rows = last_features[last_features.isna().any(axis=1)]
print(nan_rows)

# Alternatively, get a summary of NaNs in each column
nan_summary = last_features.isna().sum()
print("Summary of NaN values per column:")
print(nan_summary)


In [ ]:
last_features.head(50)

In [ ]:
missing_vessel_ids = test[~test['vesselId'].isin(last_features['vesselId'])]
print("Missing vessel IDs:", missing_vessel_ids['vesselId'].unique())

In [ ]:
### Prepare test data for predictions
expected_features = X.columns 

def prepare_test_for_predictions(test, last_features):
    test = test.copy()
    prepared_test = pd.DataFrame()

    # Create a time column in last features
    last_features['year'] = 2024
    last_features['time'] = pd.to_datetime(
        last_features[['year', 'month', 'day', 'hour', 'minute']]
    )

    # Add the columns vesselId and time
    prepared_test['vesselId'] = test['vesselId']
    prepared_test['time'] = test['time']
    print(prepared_test.head())
    # For each vessel, add the last seen features to the prepared test
    prepared_test = prepared_test.merge(last_features, on='vesselId', how='left', suffixes=('', '_last'))

    print(prepared_test.head())

    # Move the last_features to the reference row in the windows
    prepared_test['latitude_first'] = prepared_test['latitude']
    prepared_test['longitude_first'] = prepared_test['longitude']
    prepared_test['cog_sin_first'] = prepared_test['cog_sin']
    prepared_test['cog_cos_first'] = prepared_test['cog_cos']  # Fixed typo here
    prepared_test['sog_first'] = prepared_test['sog']
    prepared_test['under_way_first'] = prepared_test['under_way']
    prepared_test['time_since_start'] = (prepared_test['time'] - prepared_test['time_last']).dt.total_seconds()

    # Split the time column into month, day, hour, minute
    prepared_test['month'] = test['time'].dt.month
    prepared_test['day'] = test['time'].dt.day
    prepared_test['hour'] = test['time'].dt.hour
    prepared_test['minute'] = test['time'].dt.minute

    prepared_test['length'] = prepared_test['length']  # Ensure this column exists in 'prepared_test'
    prepared_test['breadth'] = prepared_test['breadth']  # Ensure this column exists in 'prepared_test'
    # prepared_test['DWT'] = prepared_test['DWT']  # Ensure this column exists in 'prepared_test'
    prepared_test['GT'] = prepared_test['GT']  # Ensure this column exists in 'prepared_test'
    # prepared_test['vessel_deep_sea'] = prepared_test['vessel_deep_sea']  # Ensure this column exists in 'prepared_test'
    prepared_test['rot'] = prepared_test['rot']  # Ensure this column exists in 'prepared_test'
    # prepared_test['day_of_week'] = prepared_test['time'].dt.dayofweek  # Added missing day_of_week calculation

    # Drop the columns that are no longer needed
    prepared_test.drop(['time', 'time_last', 'year'], axis=1, inplace=True)

    # Reorder the columns
    for column in expected_features:
        if column not in prepared_test.columns:
            prepared_test[column] = 0  # Assign a default value (0) for missing columns

    # Reorder columns to match expected_features
    prepared_test = prepared_test[expected_features]

    return prepared_test

# Prepare the test DataFrame
test_df = prepare_test_for_predictions(test, last_features)
test_df.head(50)

In [ ]:
predictions = best_model.predict(test_df)
print(predictions)
print(predictions.shape)

In [ ]:
predictions_df = pd.DataFrame(predictions, columns=['latitude_predicted', 'longitude_predicted'])
predictions_df['ID'] = range(len(predictions_df))
predictions_df = predictions_df[['ID', 'longitude_predicted', 'latitude_predicted']]

# Save to CSV
predictions_df.to_csv('data/submissions/predictions_7_5d_rf.csv', index=False)

In [ ]:

# Assuming 'test' and 'predictions_df' are already defined and contain the necessary data
merged_df = pd.merge(test, predictions_df, on='ID', how='left')

# Print the columns to verify
print("Columns in merged_df before dropping:", merged_df.columns.tolist())

# Drop the specified columns, checking if they exist first
columns_to_drop = ['ID', 'scaling_factor']
for col in columns_to_drop:
    if col in merged_df.columns:
        merged_df.drop(col, axis=1, inplace=True)

# Alternatively, you can drop them directly with error handling
# merged_df.drop(columns=[col for col in columns_to_drop if col in merged_df.columns], inplace=True)

# Print the columns after the drop
print("Columns in merged_df after dropping:", merged_df.columns.tolist())

# Display the first few rows of the DataFrame
print(merged_df.head())

In [ ]:
predictions_df.to_csv('data/submissions/predictions_7_5d_rf.csv', index=False)

In [ ]:
merged_df.to_csv('data/submissions/plotting_boats.csv')